# Chains

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

### Simple Chain

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template = "{country}의 수도는 무엇인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"]
)

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0
)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [7]:
response = chain.invoke(input={"country": "사우디아라비아"})

In [8]:
print(response)

사우디아라비아의 수도는 리야드(Riyadh)입니다.


In [9]:
query = "두바이의 수도는 무엇인가요?"
response = chain.invoke(query)

In [10]:
print(response)

두바이는 아랍에미리트(UAE)의 한 emirate(에미리트)이며, 두바이 자체가 수도가 아닙니다. 아랍에미리트의 수도는 아부다비(Abu Dhabi)입니다. 두바이는 UAE에서 가장 큰 도시이자 경제 중심지로 알려져 있습니다.


### Sequential Chain

- 두개 이상의 chain을 직렬 연결 처리
    - [예제] 번역 chain, 요약 chain 연결

In [12]:
from langchain_openai import OpenAI

llm = OpenAI(
    model='gpt-4o-mini',
    temperature=0.0
)

In [20]:
input_text = """
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""

trans_prompt = PromptTemplate(
    template="다음의 문장을 한글로 변역하세요:\n{text}",
    input_variables=["text"]
)

translation_chain = trans_prompt | llm | StrOutputParser()
trans_output = translation_chain.invoke(input={"text": input_text})

In [21]:
print(trans_output)

Once the data is loaded, you can use it to create a retriever that can search for relevant information based on a user's query. This retriever can then be used in conjunction with an LLM to provide more accurate and contextually relevant responses.

LLM의 한계 중 하나는 ��락 정보의 부족입니다(예: 특정 문서나 이메일에 대한 접근). 이를 해결하기 위해 LLM에 특정 외부 데이터에 대한 접근 권한을 부여할 수 있습니다.
이를 위해 먼저 문서 로더를 사용하여 외부 데이터를 로드해야 합니다. LangChain은 PDF, 이메일, 웹사이트, YouTube 비디오 등 다양한 유형의 문서에 대한 다양한 로더를 제공합니다.
데이터가 로드되면 사용자의 ���리를 기반으로 관련 정보를 검색할 수 있는 검색기를 생성하는 데 사용할 수 있습니다. 이 검색기는 LLM과 함께 사용되어 보다 정확하고 ��락에 맞는 응답을 제공할 수 있습니다.

LLM의 한계 중 하나는 ��락 정보의 부족입니다(예: 특정 문서나 이메일에 대한 접근). 이를 해결하기 위해 LLM에 특정 외부 데이터에 대한 접근 권한을 부여할 수 있습니다. 이를 위해 먼저


In [26]:
# 요약 체인
sum_prompt = PromptTemplate(
    template="다음의 글을 핵심만만 짧게 요약하세요:\n{text}",
    input_variables=["text"]
)

summary_chain = sum_prompt | llm | StrOutputParser()
summary_output = summary_chain.invoke(input={"text": trans_output})

In [27]:

print(summary_output)

 문서 로더를 사용하여 외부 데이터를 로드해야 합니다. LangChain은 PDF, 이메일, 웹사이트, YouTube 비디오 등 다양한 유형의 문서에 대한 다양한 로더를 제공합니다. 데이터가 로드되면 사용자의 ���리를 기반으로 관련 정보를 검색할 수 있는 검색기를 생성하여 LLM과 함께 사용하여 보다 정확하고 ��락에 맞는 응답을 제공할 수 있습니다.

LLM의 정보 부족 문제를 해결하기 위해 외부 데이터를 로드하고 검색기를 생성하여 사용자의 ���리에 맞는 정확한 응답을 제공할 수 있다. LangChain은 다양한 문서 유형에 대한 로더를 제공한다.


In [28]:
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(translation_chain, summary_chain)
chain

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='다음의 문장을 한글로 변역하세요:\n{text}')
| OpenAI(client=<openai.resources.completions.Completions object at 0x00000199277760C0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000019927774710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()
| PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='다음의 글을 핵심만만 짧게 요약하세요:\n{text}')
| OpenAI(client=<openai.resources.completions.Completions object at 0x00000199277760C0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000019927774710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [29]:
chain.invoke(input_text)

' LangChain은 PDF, 이메일, 웹사이트, YouTube 비디오 등 다양한 유형의 문서에 대한 다양한 로더를 제공합니다. 데이터가 로드되면 이를 사용하여 ���리를 기반으로 관련 정보를 검색할 수 있는 검색기를 생성할 수 있습니다. 이 검색기는 외부 데이터를 사용하여 LLM의 응답을 보강하는 데 사용될 수 있습니다.\n\nLLM의 정보 부족 문제를 해결하기 위해 외부 데이터를 로드하고, 이를 기반으로 ���리 검색이 가능한 검색기를 생성하여 LLM의 응답을 보강할 수 있다. LangChain은 다양한 문서 유형에 대한 로더를 제공한다.'

### Conditional Chain

In [34]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

llm = OpenAI(model='gpt-4o-mini', temperature=0.0)

# 평가 체인
grading_prompt = PromptTemplate(
    template='당신은 전문성이 있는 평가자 입니다. 아래의 답변을 1~5점으로 평가해 주세요:\n\n{text}',
    input_variables=['text']
)

grading_chain = grading_prompt | llm
# grading_chain.invoke('타이레놀은 만병 통치약입니다.')

# 질의응답 체인
default_chain = PromptTemplate(
    template='당신은 사용자의 질문에 대해 전문적인 답변을 할 수 있는 전문가 챗봇입니다.\n\n{text}',
    input_variables=['text']
)

default_chain = default_chain | llm

In [35]:
default_chain.invoke('인공지능의 최대 활용 분야는 무엇인가요?')

'\n\n인공지능(AI)의 최대 활용 분야는 다양하지만, 몇 가지 주요 분야를 ��을 수 있습니다:\n\n1. **의료**: AI는 진단, 치료 계획 수립, 환자 모니터링 및 개인 맞춤형 의료 서비스 제공에 활용됩니다. 예를 들어, 이미지 인식 기술을 통해 X-ray나 MRI 이미지를 분석하여 질병을 조기에 발견할 수 있습니다.\n\n2. **자동차**: 자율주행차 기술은 AI의 대표적인 응용 분야입니다. AI는 차량의 센서 데이터를 분석하여 주변 환경을 인식하고, 안전하게 주행할 수 있도록 ��습니다.\n\n3. **금융**: AI는 신용 평가, 사기 탐지, 투자 분석 및 고객 서비스 자동화 등 다양한 금융 서비스에 활용됩니다. 머신러닝 알고리즘을 통해 거래 패턴을 분석하고 리스크를 관리할 수 있습니다.\n\n4. **제조업**: AI는 생산 공정의 최적화, 품질 관리, 예측 유지보수 등에 사용됩니다. 이를 통해 생산성을 높이고 비용을 절감할 수 있습니다.\n\n5. **소매 및 전자상거래**: AI는 고객의 구매'

In [36]:
# 평가 분기 함수
def grading_routing_fn(input_dict) -> bool:
    # 사용자 입력이 '평가'로 시작되는지 여부 판단
    input = input_dict.get('text', '')
    return input.strip().startswith('평가')

In [37]:
cond_chain = RunnableBranch((grading_routing_fn, grading_chain), default_chain)

In [38]:
cond_chain.invoke(input={'text': 'flowith.io에서 제공하는 기능은 어떤 것인가요?'})

'\n\nflowith.io는 다양한 기능을 제공하는 플랫폼으로, 주로 팀 협업과 프로젝트 관리를 지원합니다. 주요 기능은 다음과 같습니다:\n\n1. **프로젝트 관리**: 팀원들과 함께 프로젝트를 계획하고 진행 상황을 추적할 수 있는 도구를 제공합니다.\n\n2. **작업 할당**: 팀원들에게 특정 작업을 할당하고, 각 작업의 진행 상황을 모니터링할 수 있습니다.\n\n3. **커뮤니케이션**: 팀원 간의 원활한 소통을 위한 채팅 및 댓글 기능을 제공합니다.\n\n4. **파일 공유**: 프로젝트 관련 파일을 쉽게 업로드하고 공유할 수 있는 기능이 있습니다.\n\n5. **일정 관리**: 프로젝트의 마감일과 일정을 관리할 수 있는 ��린더 기능을 제공합니다.\n\n6. **보고서 생성**: 프로젝트 진행 상황에 대한 보고서를 자동으로 생성하여 팀원들과 공유할 수 있습니다.\n\n이 외에도 flowith.io는 사용자 친화적인 인터페이스와 다양한 통합 기능을 통해 팀의 생산성을 높이는 데 도움을 ��니다. 각 기능은 팀의 필요에 맞게 조정할 수 있어 유연한 사용이 가능합니다.'

In [40]:
cond_chain.invoke(input={'text': '평가: flowith.io는 다양한 기능을 제공하는 플랫폼으로, 주로 팀 협업과 프로젝트 관리를 지원합니다. 주요 기능은 다음과 같습니다. **프로젝트 관리**: 팀원들과 함께 프로젝트를 계획하고 진행 상황을 추적할 수 있는 도구를 제공합니다. 2. **작업 할당**: 팀원들에게 특정 작업을 할당하고, 각 작업의 진행 상황을 모니터링할 수 있습니다. 3. **커뮤니케이션**: 팀원 간의 원활한 소통을 위한 채팅 및 댓글 기능을 제공합니다. 4. **파일 공유**: 프로젝트 관련 파일을 쉽게 업로드하고 공유할 수 있는 기능이 있습니다. 5. **일정 관리**: 프로젝트의 마감일과 일정을 관리할 수 있는 리마인더 기능을 제공합니다. 6. **보고서 생성**: 프로젝트 진행 상황에 대한 보고서를 자동으로 생성하여 팀원들과 공유할 수 있습니다. 이 외에도 flowith.io는 사용자 친화적인 인터페이스와 다양한 통합 기능을 통해 팀의 생산성을 높이는 데 도움을 제공합니다. 각 기능은 팀의 필요에 맞게 조정할 수 있어 유연한 사용이 가능합니다.'})

' \n\n평가 점수: 4점\n\n이유: 답변은 flowith.io의 주요 기능을 잘 설명하고 있으며, 각 기능에 대한 구체적인 설명이 포함되어 있습니다. 그러나 몇 가지 개선할 점이 있습니다. 예를 들어, 플랫폼의 장점이나 다른 유사한 플랫폼과의 차별점에 대한 언급이 부족합니다. 또한, 사용 사례나 실제 사용자 경험에 대한 정보가 포함되면 더욱 신뢰성을 높일 수 있을 것입니다. 전반적으로 유용한 정보가 제공되었지만, 추가적인 세부사항이 포함되면 더 좋을 것 같습니다.\n\n평가 점수: 4점\n\n이유: 답변은 flowith.io의 주요 기능을 잘 설명하고 있으며, 각 기능에 대한 구체적인 설명이 포함되어 있습니다. 그러나 몇 가지 개선할 점이 있습니다. 예를 들어, 플랫폼의 장점이나 다른 유사한 플랫폼과의 차별점에 대한 언급이 부족합니다. 또한, 사용 사례나 실제 사용자 경험에 대한 정보가 포함되면 더욱 신뢰성을 높일 수 있을 것입니다. 전반적으로 유용한 정보가 제공되었지만, 추가적인 세부사항이 포함되면 더 좋'

In [41]:
pass_chain = {'text': RunnablePassthrough()} | cond_chain

input_text = """
What is flowith?
Flowith is the next-generation AI creation tool. Unlike traditional chat-based AI tools, Flowith's interaction is based on a two-dimensional canvas that allows you to simultaneously interact with multiple AI models within a single workspace, enabling content generation that significantly boosts efficiency and facilitates entering a flow state during the creative process.

Flowith Oracle is our proprietary next-generation AI agent system designed to help you complete multi-step, highly complex tasks and requirements. It supports:

Autonomous planning

Task breakdown and completion

Unlimited tool invocation

Self-optimization and refinement

Flowith's Knowledge Garden is designed to let you interact with AI using your own knowledge. It automatically organizes, manages, and helps you create with your information. The system breaks down your files, notes, and online resources into small pieces of information called Seeds. Using AI, Flowith connects these Seeds to create a dynamic web of knowledge. When you chat with the AI, it automatically uses the relevant information from your Knowledge Garden, making its responses more accurate and tailored to your needs.

Compared to traditional architectures like AutoGPT, Oracle demonstrates more mature and stable performance. Additionally, by leveraging flowith's canvas-based interaction, Oracle can better present specific steps and comprehensive information to you.
"""
pass_chain.invoke(f'평가: {input_text}')

' \nOverall, Flowith represents a significant advancement in AI tools, providing a unique and efficient way to enhance creativity and productivity.\n\n평가: 4점\n\n이유: \n- 내용이 명확하고 잘 구성되어 있으며, Flowith의 기능과 장점을 잘 설명하고 있습니다.\n- 두 가지 주요 기능(Oracle과 Knowledge Garden)에 대한 설명이 구체적이고 이해하기 쉽게 작성되었습니다.\n- 그러나 Flowith의 차별점이나 경쟁 우위에 대한 구체적인 예시나 데이터가 부족하여, 독자가 더 깊이 있는 이해를 하는 데 도움이 될 수 있는 추가 정보가 필요합니다. \n- 전반적으로 ��정적인 평가를 받았지만, 더 많은 세부사항이 포함되면 좋겠습니다. \n\n결론적으로, 이 답변은 매우 유용하지만, 더 많은 구체적인 예시나 데이터를 포함하면 더욱 완벽해질 것입니다. 4점으로 평가합니다. '